In [ ]:
from datetime import datetime

def create_ts_project(s_data, segment_name):
    
    # create project
    project_name = project_name_prefix + str(segment_name)
    project = dr.Project.create(project_name=project_name, sourcedata=s_data)

    #parameters
    nbrWorkers = -1
    myMetric = 'RMSE' 

    # Set up KIA features
    feature_settings = []
    kia_columns = kia_vars_list
    dnd_columns = do_not_derive
    for column in kia_columns:
        feature_settings.append(dr.FeatureSettings(column, known_in_advance=True))

    feature_settings += [dr.FeatureSettings(feat_name, do_not_derive=True) for feat_name in dnd_columns]
                    
    # spec
    mySpec = dr.DatetimePartitioningSpecification(
                datetime_partition_column=date_col,
                use_time_series=True,
                multiseries_id_columns=['multiseries_id'],

                default_to_known_in_advance=False,
                feature_settings=feature_settings, 

                feature_derivation_window_start=feature_derivation_window_start,
                feature_derivation_window_end=feature_derivation_window_end,

                forecast_window_start=forecast_window_start,
                forecast_window_end=forecast_window_end,
        
                validation_duration=validation_duration,

                #holdout_start_date=holdout_start_date,
                #holdout_duration=holdout_duration,
        
                use_cross_series_features = True,
                aggregation_type = 'total',
                cross_series_group_by_columns = ['nameplate'],

                number_of_backtests = number_of_backtests,
                #differencing_method=DIFFERENCING_METHOD.SEASONAL,
                #periodicities=[dr.Periodicity(time_steps=7, time_unit=TIME_UNITS.DAY)],
  
                calendar_id=myCal.id
        )
    
    advanced_options = dr.AdvancedOptions(
        #weights='weights_column',
        #offset=['offset_column'],
        #exposure='exposure_column',
        #response_cap=0.7,
        blueprint_threshold=1440,
        #smart_downsampled=True, 
        #majority_downsampling_rate=75.0
        accuracy_optimized_mb=True
    )
    
    project.set_target(
                        target = target_variable_name,
                        metric = myMetric,
                        partitioning_method = mySpec,
                        worker_count= nbrWorkers,
                        advanced_options=advanced_options,
                        mode=dr.AUTOPILOT_MODE.MANUAL  ##### Here is where we can change to dr.AUTOPILOT_MODE.FULL_AUTO
                      )

    return project.id

In [ ]:
model_loop_list = dfMod[segment_to_model_foreach].unique()

for s in list(set(model_loop_list)):
    
    # this line is for when you're partially done and you hit a timeout error
    if s in ['segment_1','segment_4','segment_5','segment_2','segment_3']:
        continue
    
    # filter data
    data_to_send = traindata.loc[traindata[segment_to_model_foreach]==str(s)]
    
    # create a custom specs
    number_of_backtests = 2
    
    validation_duration = 'P120D'
    
    #holdout_start_date = pd.to_datetime("2013-11-01")
    holdout_duration='P120D'
    
    forecast_window_start=1
    forecast_window_end=31
    
    # fdw settings
    feature_derivation_window_start = -7
    feature_derivation_window_end = 0
        
    # submit the model to DataRobot
    pid = create_ts_project(data_to_send, str(s))
    
    p_list.append(pid)
    
    print(s, sep='', end='...Done! | ', flush=True)

print("All projects are now submitted!")